# Land Cover Classification Values, Count, and Update

In this notebook, there are three tasks: 
- How to get discrete/unique values from a raster layer
- How to count the number of pixels in each discrete/unique value
- How to update the values of a raster layer with another. This code is particularly useful when wanting to update a part of the values of a raster layer, not the entire layer.  

In [1]:
# Import libraries
import rasterio
import numpy as np
from rasterio.plot import show
import time

---
### Land Cover Classification: Unique Classes

In this section, we are wanting to get all discrete/unique classes in a raster layer. The first section, `function`, includes the function created to get all unique values, and the `input` section is where you input your raster layer of interest and print all discrete/unique values.

#### Function

In [2]:
# The function which will return the unique classes in that file
def get_unique_class_values(raster_path):
    with rasterio.open(raster_path) as src:
        raster_data = src.read(1)
        nodata_value = src.nodata

        unique_values = np.unique(raster_data[raster_data != nodata_value])
        return unique_values

#### Input

In [ ]:
%%time
# Replace with your file path
raster_file = 'file_path_to_your_raster_file_here.tif'

# Get unique class values from the LCA raster
unique_class_values = get_unique_class_values(raster_file)

# Print the unique class values
print("Unique Class Values:", unique_class_values)

---
### Land Cover Classification: Count Pixels in each Class

Now that we have all discrete/unique values, this section now counts the number of pixels in each class/value. To do this, we first create a function in the `function` section, and then apply to function to our data in the `input` section.

#### Function

In [3]:
# The function which can count the pixel count for each class
def count_pixels_by_class(raster_path, class_values):
    with rasterio.open(raster_path) as src:
        raster_data = src.read(1)
        nodata_value = src.nodata

        pixel_counts = {}
        
        for class_value in class_values:
            if class_value != nodata_value:
                count = np.sum(raster_data == class_value)
                pixel_counts[class_value] = count

        return pixel_counts

#### Input

In [ ]:
# First, define the class values that you want to count. These values will most likely be obtained from the previous step

# Define class values to count
class_values_to_count = [a, list, of, values, here, e.g., 1, 2, 3, 4]  # Modify as needed

In [ ]:
%%time
# Set the file path to your raster file
raster_file = 'file_path_to_your_raster_file_here.tif'

# Count pixels in each class within the LCA raster
pixel_counts = count_pixels_by_class(raster_file, class_values_to_count)

# Print the pixel counts
for class_value, count in pixel_counts.items():
    print(f"Class {class_value}: {count} pixels")

## Update Raster Layer with Values From Another Raster Layer 

The purpose of this code is to be able to update the values in a raster layer, with another raster layer which has a different extent. In other words, the values of the original raster layer (LCA) will only update when values in the new raster layer (LCB) are present.

To be able to do this, the extent of both layers must be the same (which sounds like it defeats the object of the section). However, this can be achieved by filling the partial layer with no data values where there is no information. This can be done in QGIS - export the raster layer and set the layer extent by the original raster layer (LCA).  

In [ ]:
%%time
# Load the original raster layer (LCA) and new raster layer (LCB) 
with rasterio.open('your_original_raster_layer.tif') as src_lca, rasterio.open('your_new_raster_layer.tif') as src_lcb:
    
    # Read the data from LCB
    lcb_data = src_lcb.read(1)

    # Read the data from LCA and update LCA pixel values where LCB has existing (i.e. non-null) values
    lca_data = src_lca.read(1)
    lca_data[lcb_data != src_lcb.nodata] = lcb_data[lcb_data != src_lcb.nodata]

    # Update metadata
    profile = src_lca.profile

    # Write the updated LCA data to a new raster
    with rasterio.open('file_path_to_new_raster_layer.tif', 'w', **profile) as dst:
        dst.write(lca_data, 1)

#### Example of update on multiple conditions

The first section updates the all pixel values in LCA if pixel values exist in LCB. This section is an example of if we only want to update the values in LCA based on a condition or multiple conditions. For example, in this section, we want to change the value of pixels in the original raster layer (LCA) to 51 **if** the pixel value is equal to `50` in the original raster layer, and equal to `4` in the new raster layer. 

In [ ]:
%%time
# Load the original raster layer (LCA) and new raster layer (LCB) 
with rasterio.open('your_original_raster_layer.tif') as src_lca, rasterio.open('your_new_raster_layer.tif') as src_lcb:
    
    # Read the data from LCB
    lcb_data = src_lcb.read(1)

    # Update LCA pixel values to 51 where LCB is equal to coniferous forest (4) and the original layer is equal to trees (50)
    lca_data = src_lca.read(1)
    lca_data[(lcb_data == 4) & (lca_data == 50)] = 51

    # Update metadata if necessary
    profile = src_lca.profile

    # Write the updated LCA data to a new raster
    with rasterio.open('file_path_to_new_raster_layer.tif', 'w', **profile) as dst:
        dst.write(lca_data, 1)